In [1]:
# Sarah loves music

from carabao.neurotron import Pulse,Terminal,Synapses,toy
from carabao.util import repr
from numpy import array

#===================================================================================
# class: Core
#===================================================================================

class Core:
    """
    class Core: implements a Neurotron core functionality
    >>> core = Core(k,excite,predict,depress,(10,3))
    """
    def __init__(self,k, e,d,p, name=""):
        self.k = k
        self.name = name

        self.excite  = Terminal(e.w[k],e.theta,'excite')
        self.depress = Terminal(d.w[k],d.theta,'depress')
        self.predict = Terminal(p.W[k],p.theta,'predict')

    def feed(self,f,c,V,log=None):
        e = self.excite.empower(f)
        u = self.excite.spike(e)
        if log is not None: 
            print(log,self.name,"e:",repr(e), "u:",u)

        v = self.depress.empower(c)
        d = self.depress.spike(v) 
        if log is not None: 
            print(log,self.name,"v:",repr(v), "d:",d)

        E = self.predict.empower(V)
        s = self.predict.spike(E)
        if log is not None: 
            print(log,self.name,"E:",repr(E), "s:",repr(s))

    def __repr__(self):
        return "Core('%s',%g"% (self.name,self.k) + ")"
        
#===================================================================================
# class: Neurotron
#===================================================================================

class Neurotron:
    """
    class Neurotron: full functionality
    >>> par = toy('sarah')
    >>> cell0 = Neurotron(k:=0,par,(10,3),'cell0')
    >>> print(cell0)
    """
    def __init__(self,k,par,partition=None,name=None):
        self.k = k
        self.partition = partition
        self.name = name
        
        excite,depress,predict = par
        
        syn = Synapses(excite.k[k],excite.p[k],excite.eta,'syn-e')
        self.excite  = Terminal(excite.w[k],excite.theta,syn,'excite')

        syn = Synapses(depress.g[k],depress.p[k],depress.eta,'syn-d')
        self.depress = Terminal(depress.w[k],depress.theta,syn,'depress')
    
        syn = Synapses(predict.K[k],predict.P[k],predict.eta,'syn-p')
        self.predict = Terminal(predict.W[k],predict.theta,syn,'predict')

    def feed(self,f,c,x):
        e = self.excite.empower(f)
        u = self.excite.spike(e)
        print("e:",repr(e), "u:",u)

        v = self.depress.empower(c)
        d = self.depress.spike(v) 
        print("v:",repr(v), "d:",d)

        #E = self.predict.empower(V)
        #s = self.predict.spike(E)
        E,s = self.predict.feed(x)
        print("E:",repr(E), "s:",repr(s))
     
    def __repr__(self):
        return "Neurotron('%s',%g"% (self.name,self.k) + ")"

In [22]:
# working with terminals

par,token = toy('sarah');  epar,dpar,ppar = par
excite  = Terminal(epar.w[0],ppar.theta,None,'excite')
depress = Terminal(dpar.w[0],dpar.theta,None,'depress')
predict = Terminal(ppar.W[0],ppar.theta,None,'predict')

f=[1,1,0,1,1,1,0,1,0,1]
c=[1,1,1];  V = [c,c]

e = excite.empower(f)
u = excite.spike(e)
print("e:",repr(e), "u:",u)

v = depress.empower(c)
d = depress.spike(v) 
print("v:",repr(v), "d:",d)

E = predict.empower(V)
S = predict.spike(E)
print("E:",repr(E), "S:",repr(S))




e: #[1 1 0 1 1 1 0 1 0 1] u: [1]
v: #[1 1 0] d: [1]
E: #[1 0 0; 0 1 1] S: :[0 1]


In [23]:
# working with a Neurotron core

par,token = toy('sarah');  epar,dpar,ppar = par
excite  = Terminal(epar.w[0],ppar.theta,'excite')
depress = Terminal(dpar.w[0],dpar.theta,'depress')
predict = Terminal(ppar.W[0],ppar.theta,'predict')

core = Core(0,epar,dpar,ppar,'core0')

f=[1,1,0,1,1,1,0,1,0,1]
c=[1,1,1];  V = [c,c]

e = core.excite.empower(f)
u = core.excite.spike(e)
print("e:",repr(e), "u:",u)

E = core.predict.empower(V)
s = core.predict.spike(E)
print("E:",repr(E), "s:",repr(s))

v = core.depress.empower(c)
d = core.depress.spike(E) 
print("v:",repr(v), "d:",d)


e: #[1 1 0 1 1 1 0 1 0 1] u: [1]
E: #[1 0 0; 0 1 1] s: :[0 1]
v: #[1 1 0] d: [1 1]


In [24]:
dpar.p


[[1, 1, 1], [1, 1, 1], [1, 1, 1]]

In [14]:
# feeding the Neurotron core

par,token = toy('sarah');
core = Core(0,epar,ppar,dpar) 

f=[1,1,0,1,1,1,0,1,0,1];  c=[1,1,1]; 
#core.feed(f,c,f+c)

ValueError: not enough values to unpack (expected 4, got 2)

In [15]:
e = core.excite.empower(token['loves'])
u = core.excite.spike(e)
(repr(e),repr(u))

('#[0 1 0 1 0 1 0 0 0 1]', ':[0]')

In [16]:
K = [[10,11,12],[10,11,12]];  P = [[.5,.4,.1],[.6,.2,.3]];  eta = 0.5
syn = Synapses(K,P,eta,log='Synapses:')
V = syn.feed(x:=[0,0,0,0,0,0,0,0,0,0,1,1,0])

Synapses: [0 0 0 0 0 0 0 0 0 0 1 1 0]  ->  #[1 0 0; 1 0 0]


In [17]:
# Neurotron core with Synapses

f = [1,1,0,1,1,1,0,1,0,1];  c = [1,1,1]; x = f+c
epar,dpar,ppar,token = toy('sarah')
#core = Core(0,epar,dpar,ppar,[len(f),len(c)],'core0') 
core = Core(0,epar,dpar,ppar,'core0') 
print(core)
#core.feed(x[:10],x[10:13],x)

ValueError: not enough values to unpack (expected 4, got 2)

In [18]:
# Neurotron example

f = [1,1,0,1,1,1,0,1,0,1];  c = [1,1,1]; x = f+c
e,d,p,token = toy('sarah')
cell = Neurotron(0,(e,d,p),(len(f),len(c)),'cell0') 
print(cell,cell.excite)
#core.feed(x[:10],x[10:13],x)

ValueError: not enough values to unpack (expected 4, got 2)

In [19]:
#=========================================================================
# nice demo
#=========================================================================
f = [1,1,0,1,1,1,0,1,0,1];  c = [1,1,1]; x = f+c
par,token = toy('sarah')
cell = Neurotron(0,par,(len(f),len(c)),'cell0') 

print(cell.excite)
E = cell.excite.empower(token['Sarah'],"empower 'Sarah':")
s = cell.excite.spike(E,'         spike:')

E = cell.excite.empower(token['loves'],"empower 'loves':")
s = cell.excite.spike(E,'         spike:')

Terminal('excite',#[1 1 0 1 1 1 0 1 0 1],6) @ {#[0 1 2 3 4 5 6 7 8 9], #[1 1 0 1 1 1 0 1 0 1] @ 0.5}
empower 'Sarah': [1 1 0 1 1 1 0 1 0 1]  ->  #[1 1 0 1 1 1 0 1 0 1]
         spike: #[1 1 0 1 1 1 0 1 0 1]  ->  :[1]
empower 'loves': [0 1 1 1 0 1 1 0 1 1]  ->  #[0 1 0 1 0 1 0 0 0 1]
         spike: #[0 1 0 1 0 1 0 0 0 1]  ->  :[0]


In [11]:
par[0].w



[[1, 1, 0, 1, 1, 1, 0, 1, 0, 1],
 [0, 1, 1, 1, 0, 1, 1, 0, 1, 1],
 [1, 1, 1, 0, 0, 1, 0, 1, 1, 1]]